In [59]:
import vtk 
vtk.vtkLogger.SetStderrVerbosity(vtk.vtkLogger.VERBOSITY_OFF)
import plotly.io as pio
pio.renderers.default = "notebook+pdf"

: 

# 1. Getting Started

A verse scenario is defined by a map, a set of agents and, if there exist multiple agents, a sensor. 

In this section, we are going to look at how to create a simple scenario in Verse. In this scenario we will have a drone following a straight track and will avoid by moving upwards to avoid a static obstacle at known position in the track. 

## 1.1 Instantiate map
The map of a scenario specifies the tracks that the agents can follow. In this example, we will only look at a simple map with two types of tracks: 1) type <code>T0</code>, which describe straight track that align with x-axis and 2) type <code>TAvoidUp</code>, which describes set of tracks with upward direction that the agent can follow while avoiding the obstacle. The types of tracks will also be referred as track modes in later sections. We will discuss in detail how to create maps in later sections. In this section we will import a pre-defined map. 

In [60]:
from tutorial_map import M3

map1 = M3()

## 1.2 Creating agent
To create such a scenario in Verse, we first need to create an agent for Verse. An agent in Verse is defined by a set of tactical modes, a decision logic to determine the transition between tactical modes, and a flow function that defines continuous evolution. The agent's tactical mode and decision logic are provided as python code strings and the flow function is provided as a python function. 

The tactical mode of the agents corresponds to an agent's decision. For example, in this drone avoidance example, the tactical mode for the agent can be <code>Normal</code> and <code>AvoidUp</code>. The decision logic also need to know the available track modes from the map. The tactical modes and track modes are provided as <code>Enums</code> to Verse.  

In [61]:
from enum import Enum, auto


class CraftMode(Enum):
    Normal = auto()
    AvoidUp = auto()


class TrackMode(Enum):
    T0 = auto()
    TAvoidUp = auto()

We also require the user to provide the continuous and discrete variables of the agents together with the decision logic. The variables are provided inside class with name <code>State</code>. Variables end with <code>_mode</code> will be identify by verse as discrete variables. In the example below, <code>craft_mode</code> and <code>track_mode</code> are the discrete variables and <code>x</code>, <code>y</code>, <code>z</code>, <code>vx</code>, <code>vy</code>, <code>vz</code> are the continuous variables. The type hints for the discrete variables are necessary to associate discrete variables with the tactical modes and lane modes defined above

In [62]:
class State:
    x: float
    y: float
    z: float
    vx: float
    vy: float
    vz: float
    craft_mode: CraftMode
    track_mode: TrackMode

    def __init__(self, x, y, z, vx, vy, vz, craft_mode, track_mode):
        pass

The decision logic describe for an agent takes as input its current state and the (observable) states of the other agents if there's any, and updates the tactical mode of the ego agent. In this example, the decision logic is traight forward: When the x position of the drone is close to the obstacle (20m), the drone will start moving upward. There's no other agents in this scenario. The decision logic of the agent can be written in an expressive subset of Python inside function <code>decisionLogic</code>. 

In [63]:
import copy


def decisionLogic(ego: State, track_map):
    next = copy.deepcopy(ego)
    if ego.craft_mode == CraftMode.Normal:
        if ego.x > 20:
            next.craft_mode = CraftMode.AvoidUp
            next.track_mode = track_map.h(ego.track_mode, ego.craft_mode, CraftMode.AvoidUp)
    return next

We incoperate the above definition of tactical modes and decision logic into code strings and combine it with an imported agent flow, we can then obtain the agent for this sceanrio. 

In [64]:
from tutorial_agent import DroneAgent

drone1 = DroneAgent("drone1", file_name="dl_sec1.py", t_v_pair=(1, 1), box_side=[0.4] * 3)

## 1.3 Creating scenario
With the agent and map defined, we can now define the scenario.

In [65]:
from verse.scenario import Scenario

scenario = Scenario()

We can set the initial condition of the agent and add the agent 

In [66]:
drone1.set_initial(
    [[0, -0.5, -0.5, 0, 0, 0], [1, 0.5, 0.5, 0, 0, 0]], (CraftMode.Normal, TrackMode.T0)
)
scenario.add_agent(drone1)

and set the map for the sceanrio

In [67]:
scenario.set_map(map1)

Since we only have one agent in the scenario, we don't need to specify a sensor. 

We can then compute simulation traces or reachable states for the scenario

In [68]:
traces_simu = scenario.simulate(60, 0.2)
traces_veri = scenario.verify(60, 0.2)

We can visualize the results using functions provided with Verse

In [ ]:
%pip install nbformat
import sys
print("Python executable:", sys.executable)
import nbformat
print("nbformat version:", nbformat.__version__)


In [ ]:
from verse.plotter.plotter3D import *

import pyvista as pv
import warnings

warnings.filterwarnings("ignore")
from verse.plotter.plotter2D import *
import plotly.graph_objects as go

# 设置 Plotly 渲染器为浏览器
pio.renderers.default = "browser"

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 1, 3)
fig.show()

pv.set_jupyter_backend(None)
fig = pv.Plotter()
fig = plot3dMap(map1, ax=fig)
fig = plot3dReachtube(traces_veri, "drone1", 1, 2, 3, color="r", ax=fig)
fig.set_background("#e0e0e0")
fig.show()

# 2. Adding multiple agents

In the previous section, we played around with a scenario with a single drone. In this example, we will create a scenario with multiple agents and explore the potential of Verse to handle multi-agent scenarios. 

In this example, we will look at a two drone collision avoidance example. In this example, we have two drones running in a map with three parallel tracks arranged in vertical directions. The two drones are running in the same direction on the middle track. However, the later drone is moving faster than the drone in the front and it will perform a track switch while getting close to the front drone to avoid collision. 

We will first setup a new scenario and add the map.

In [71]:
from verse.scenario import Scenario
from tutorial_map import M4

scenario = Scenario()
scenario.set_map(M4())

In [72]:
from enum import Enum, auto


class CraftMode(Enum):
    Normal = auto()
    MoveUp = auto()
    MoveDown = auto()


class TrackMode(Enum):
    T0 = auto()
    T1 = auto()
    T2 = auto()
    M01 = auto()
    M10 = auto()
    M12 = auto()
    M21 = auto()


class State:
    x: float
    y: float
    z: float
    vx: float
    vy: float
    vz: float
    craft_mode: CraftMode
    track_mode: TrackMode

    def __init__(self, x, y, z, vx, vy, vz, craft_mode, track_mode):
        pass

Verse's decision logic support python <code>any</code> and <code>all</code> functions, which allows the decision logic to quantify over other agents in the scenario. This enable user to easily create multi-agent scenario. In addition, the support of user defined functions, such as the <code>is_close</code> function in example, enable user to write more complicated decision logic. In this case, the decision logic will take an additional argument <code>others</code>, which provides the states for other agents. Notice the type hint for both <code>ego</code> and <code>others</code>. The updated agent's decision logic looks like following.

In [74]:
from typing import List
import copy


def is_close(ego, other):
    res = (
        (other.x - ego.x < 10 and other.x - ego.x > 8)
        or (other.y - ego.y < 10 and other.y - ego.y > 8)
        or (other.z - ego.z < 10 and other.z - ego.z > 8)
    )
    return res


def decisionLogic(ego: State, others: List[State], track_map):
    next = copy.deepcopy(ego)

    if ego.craft_mode == CraftMode.Normal:
        if any(
            ego.x < other.x and (is_close(ego, other) and ego.track_mode == other.track_mode)
            for other in others
        ):
            if track_map.h_exist(ego.track_mode, ego.craft_mode, CraftMode.MoveUp):
                next.craft_mode = CraftMode.MoveUp
                next.track_mode = track_map.h(ego.track_mode, ego.craft_mode, CraftMode.MoveUp)
            if track_map.h_exist(ego.track_mode, ego.craft_mode, CraftMode.MoveDown):
                next.craft_mode = CraftMode.MoveDown
                next.track_mode = track_map.h(ego.track_mode, ego.craft_mode, CraftMode.MoveDown)

    if ego.craft_mode == CraftMode.MoveUp:
        if (
            track_map.altitude(ego.track_mode) - ego.z > -1
            and track_map.altitude(ego.track_mode) - ego.z < 1
        ):
            next.craft_mode = CraftMode.Normal
            if track_map.h_exist(ego.track_mode, ego.craft_mode, CraftMode.Normal):
                next.track_mode = track_map.h(ego.track_mode, ego.craft_mode, CraftMode.Normal)

    if ego.craft_mode == CraftMode.MoveDown:
        if (
            track_map.altitude(ego.track_mode) - ego.z > -1
            and track_map.altitude(ego.track_mode) - ego.z < 1
        ):
            next.craft_mode = CraftMode.Normal
            if track_map.h_exist(ego.track_mode, ego.craft_mode, CraftMode.Normal):
                next.track_mode = track_map.h(ego.track_mode, ego.craft_mode, CraftMode.Normal)

    return next

With the updated decision logic, we can now spawn the two agents with their initial conditions. 

In [75]:
from tutorial_agent import DroneAgent

drone1 = DroneAgent("drone1", file_name="dl_sec2.py", t_v_pair=(1, 1), box_side=[0.4] * 3)
drone1.set_initial(
    [[1.5, -0.5, -0.5, 0, 0, 0], [2.5, 0.5, 0.5, 0, 0, 0]], (CraftMode.Normal, TrackMode.T1)
)

drone2 = DroneAgent("drone2", file_name="dl_sec2.py", t_v_pair=(1, 0.5), box_side=[0.4] * 3)
drone2.set_initial(
    [[19.5, -0.5, -0.5, 0, 0, 0], [20.5, 0.5, 0.5, 0, 0, 0]], (CraftMode.Normal, TrackMode.T1)
)

We can then add both agents to the scenario

In [76]:
scenario.add_agent(drone1)
scenario.add_agent(drone2)

With multiple agents in the scenario, we now need to add the sensor. The sensor defines how an agent is visible to other agents. In this example, we will use the default sensor function, which allows all agents to see all variables of other agents. 

In [77]:
from tutorial_sensor import DefaultSensor

scenario.set_sensor(DefaultSensor())

With the scenario fully constructed, we can now simulte or verify the scenario using the simulate/verify function provided by Verse. 

In [ ]:
traces_simu = scenario.simulate(60, 0.2)
traces_veri = scenario.verify(60, 0.2)

We can visualize the results using functions provided with Verse

In [ ]:
from verse.plotter.plotter3D import *
import pyvista as pv
import warnings

warnings.filterwarnings("ignore")

from verse.plotter.plotter2D import *
import plotly.graph_objects as go

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 1, 3)
fig.show()

pv.set_jupyter_backend(None)
fig = pv.Plotter()
fig = plot3dMap(M4(), ax=fig)
fig = plot3dReachtube(traces_veri, "drone1", 1, 2, 3, color="r", ax=fig)
fig = plot3dReachtube(traces_veri, "drone2", 1, 2, 3, color="b", ax=fig)
fig.set_background("#e0e0e0")
fig.show()

# 3. Adding safety assertions

In [46]:
from enum import Enum, auto


class CraftMode(Enum):
    Normal = auto()
    MoveUp = auto()
    MoveDown = auto()


class TrackMode(Enum):
    T0 = auto()
    T1 = auto()
    T2 = auto()
    M01 = auto()
    M10 = auto()
    M12 = auto()
    M21 = auto()


class State:
    x: float
    y: float
    z: float
    vx: float
    vy: float
    vz: float
    craft_mode: CraftMode
    track_mode: TrackMode

    def __init__(self, x, y, z, vx, vy, vz, craft_mode, track_mode):
        pass

Verse allow checking safety conditions while performing simulation and verification. The safety conditions in verse can be specified using Python assert statements in the decision logic. With the <code>any</code> and <code>all</code> functions, the user can also define safety conditions between different agents. In this example, we are going to add two safety conditions: 1) the distance between two agents in all x,y,z direction should always be greater than or equal to 1.0m and 2) both drones should never enter region $40\leq x\leq 50$, $-5\leq y\leq 5$, $-10\leq z\leq -6$. The two safety asserts in the <code>decisionLogic</code> is shown below. 

In [47]:
from typing import List
import copy


def decisionLogic(ego: State, others: List[State], track_map):
    next = copy.deepcopy(ego)
    """
    """

    assert not any(
        ego.x - other.x < 1
        and ego.x - other.x > -1
        and ego.y - other.y < 1
        and ego.y - other.y > -1
        and ego.z - other.z < 1
        and ego.z - other.z > -1
        for other in others
    ), "Safe Seperation"

    assert not (
        ego.x > 40 and ego.x < 50 and ego.y > -5 and ego.y < 5 and ego.z > -10 and ego.z < -6
    ), "Unsafe Region"

    return next

We can then construct the two agents and the scenario exactly as previous section.

In [48]:
from verse.scenario import Scenario
from tutorial_map import M4

scenario = Scenario()
scenario.set_map(M4())

from tutorial_agent import DroneAgent

drone1 = DroneAgent("drone1", file_name="dl_sec3.py", t_v_pair=(1, 1), box_side=[0.4] * 3)
drone1.set_initial(
    [[1.5, -0.5, -0.5, 0, 0, 0], [2.5, 0.5, 0.5, 0, 0, 0]], (CraftMode.Normal, TrackMode.T1)
)
scenario.add_agent(drone1)

drone2 = DroneAgent("drone2", file_name="dl_sec3.py", t_v_pair=(1, 0.5), box_side=[0.4] * 3)
drone2.set_initial(
    [[19.5, -0.5, -0.5, 0, 0, 0], [20.5, 0.5, 0.5, 0, 0, 0]], (CraftMode.Normal, TrackMode.T1)
)
scenario.add_agent(drone2)

from tutorial_sensor import DefaultSensor

scenario.set_sensor(DefaultSensor())

We can then simulate and verify the scenario and visualize the result. We can see from the result that drone1 enters the unsafe region, which causes a safety condition violation.

In [ ]:
traces_simu = scenario.simulate(60, 0.2)
traces_veri = scenario.verify(60, 0.2)

In [ ]:
from verse.plotter.plotter3D import *
import pyvista as pv
import warnings

warnings.filterwarnings("ignore")

from verse.plotter.plotter2D import *
import plotly.graph_objects as go

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 1, 3)
fig.show()

pv.set_jupyter_backend(None)
fig = pv.Plotter()
fig = plot3dMap(M4(), ax=fig)
fig = plot3dReachtube(traces_veri, "drone1", 1, 2, 3, color="r", ax=fig)
fig = plot3dReachtube(traces_veri, "drone2", 1, 2, 3, color="b", ax=fig)
fig.set_background("#e0e0e0")
fig.show()

# 4. Creating agent flow function

In previous sections, we discussed about how to create the decision logic for agents in Verse, in section we will look into detail about how the agent flows are specified and how the pieces are combined together for an agent in Verse. 

In this example, we will look at a simple kinetic bicycle model. The dynamics equations of the car are given by 
$$\dot{x} = v cos(\theta)$$
$$\dot{y} = v sin(\theta)$$
$$\dot{\theta} = \frac{v}{L}*tan(\delta)$$
$$\dot{v} = a$$
which are implemented as in the <code>car_dynamics</code> function below

In [49]:
import numpy as np


def car_dynamics(t, state, u):
    x, y, theta, v = state
    delta, a = u
    x_dot = v * np.cos(theta + delta)
    y_dot = v * np.sin(theta + delta)
    theta_dot = v / 1.75 * np.tan(delta)
    v_dot = a
    return [x_dot, y_dot, theta_dot, v_dot]

The flow function define the continuous time evolution of agents' continuous states. It takes as input the initial states of the agent and the a time, and outputs the state of the agent from that initial states at that time. In Verse, the flow function is implemented by the <code>TC_simulate</code> function in the agent class. The <code>TC_simulate</code> takes as input a mode, a initial continuous states, a time bound, a simulation time step and the map. It will output simulation of agent dynamics at given mode starting from the initial continuous states, until the time bound with time step as an np array. Below shows an implementation of the <code>TC_simulate</code> function for the car agent. Note some detail helper functions for the car dynamics placed in <code>tutorial_utils</code> are not shown explicitly in the example below. 

In [50]:
from tutorial_utils import car_action_handler
from typing import List
import numpy as np
from scipy.integrate import ode

def TC_simulate(
    mode: List[str], init, time_bound, time_step, lane_map: LaneMap = None
):
    time_bound = float(time_bound)
    num_points = int(np.ceil(time_bound / time_step))
    trace = np.zeros((num_points + 1, 1 + len(init)))
    trace[1:, 0] = [round(i * time_step, 10) for i in range(num_points)]
    trace[0, 1:] = init
    for i in range(num_points):
        steering, a = car_action_handler(mode, init, lane_map)
        r = ode(car_dynamics)
        r.set_initial_value(init).set_f_params([steering, a])
        res: np.ndarray = r.integrate(r.t + time_step)
        init = res.flatten()
        if init[3] < 0:
            init[3] = 0
        trace[i + 1, 0] = time_step * (i + 1)
        trace[i + 1, 1:] = init
    return trace

With the <code>TC_simulate</code> function specified, we can now combine all parts together to construct a new car agent. An agent class for Verse should have four attributes: 1) <code>id</code>: an unique identifier for each agent in a scenario, 2) <code>controller</code>: a <code>ControllerIR</code> object, which is an intermediate representation of decisionLogic in Verse. It can be constructed automatically by passing in the decision logic code strings described in previous sections, 3) <code>TC_simulate</code> function, and 4) functions for setting initial condition of the agent, which can be inherited from <code>BaseAgent</code> class. 

In [51]:
from verse.parser.parser import ControllerIR
from verse.agents import BaseAgent


class CarAgent(BaseAgent):
    def __init__(self, id, code=None, file_name=None):
        self.id = id
        self.decision_logic = ControllerIR.parse(code, file_name)
        self.TC_simulate = TC_simulate

We are going to test this CarAgent in a two car scenario with simple braking decision logic in an one track map. We can setup the scenario in Verse as we discussed in the previous section and simulate/verify the scenario. The results are visualized below. 

In [ ]:
from enum import Enum, auto


class AgentMode(Enum):
    Normal = auto()
    Brake = auto()


class TrackMode(Enum):
    T0 = auto()


from verse.scenario import Scenario
from tutorial_map import M1

scenario = Scenario()
scenario.set_map(M1())


car1 = CarAgent("car1", file_name="dl_sec4.py")
car1.set_initial([[0, -0.5, 0, 2], [1, 0.5, 0, 2]], (AgentMode.Normal, TrackMode.T0))
car2 = CarAgent("car2", file_name="dl_sec4.py")
car2.set_initial([[15, -0.5, 0, 1], [16, 0.5, 0, 1]], (AgentMode.Normal, TrackMode.T0))
scenario.add_agent(car1)
scenario.add_agent(car2)

traces_simu = scenario.simulate(10, 0.01)
traces_veri = scenario.verify(10, 0.01)

import plotly.graph_objects as go
from verse.plotter.plotter2D import *

fig = go.Figure()
fig = simulation_tree(traces_simu, None, fig, 0, 1, [0, 1], "lines", "trace")
fig.show()

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 0, 1, [0, 1], "lines", "trace")
fig.show()

# 5. Creating map

In this section, we will look at how to create a map for Verse by extending the map we used in the previous section from 1 lane to 2 lanes. 

The map will contain a set of tracks, that the agent's continuous dynamics roughly follows. In addition, we also assume that an agent's decision logic does not depend on exactly which of the infinitely many tracks it is following, but instead, it depends only on which type of track it is following, or the track mode. In this example, as shown in the figure below, each of the lanes will have a track mode <code>T0</code> and <code>T1</code>. In Verse, we can create the two types of tracks <code>T0</code> and <code>T1</code> using the <code>Lane</code> object, with each of them having one straight segment constructed by <code>StraightLane</code> as shown below. 


In [53]:
from verse.map.lane_segment import StraightLane
from verse.map.lane import Lane

segment0 = StraightLane("seg0", [0, 0], [500, 0], 3)
lane0 = Lane("T0", [segment0])
segment1 = StraightLane("seg1", [0, 3], [500, 3], 3)
lane1 = Lane("T1", [segment1])

Verse currently implments <code>StraightLane</code> and <code>CircularLane</code>, which corresponding to straight line and circular curves. In addition, we allow each <code>Lane</code> to have multiple concatenated <code>StraightLane</code> or <code>CircularLane</code>.

In addition, we also define the transition between tracks. For example, every track for transitioning from point on <code>T0</code> to a corresponding point on <code>T1</code> has track mode <code>M01</code>. Therefore, in this example, our map will have a total of 4 track modes <code>T0</code>, <code>T1</code>, <code>M01</code>, and <code>M10</code>. 

![map_2lanes_lab](map_2lanes_lab.png)


The map will have a mapping function <code>g</code>, which can provide the track that the agent is following given a track mode and the agent's current position. In this example, we can inherit the g function from the base class <code>LaneMap</code>

Finally, a Verse agent's decision logic can change its interal or tactical mode (E.g. <code>Normal</code> to <code>SwitchLeft</code>). When an agent changes its tactical mode, it may also update the track it's following and this is encoded in another function <code>h</code>, which takes the current track mode, the current and the next tactical mode, and generates the new track mode the agent should follow. In this example, the agents have tacical mode <code>Normal</code>, <code>SwitchLeft</code> and <code>SwitchRight</code>. Therefore, an example <code>h</code> function looks like following. 

In [54]:
h_dict = {
    ("T0", "Normal", "SwitchLeft"): "M01",
    ("T1", "Normal", "SwitchRight"): "M10",
    ("M01", "SwitchLeft", "Normal"): "T1",
    ("M10", "SwitchRight", "Normal"): "T0",
}


def h(lane_idx, agent_mode_src, agent_mode_dest):
    return h_dict[(lane_idx, agent_mode_src, agent_mode_dest)]


def h_exist(lane_idx, agent_mode_src, agent_mode_dest):
    return (lane_idx, agent_mode_src, agent_mode_dest) in h_dict

With all these pieces, we can now create the map for this scenario.

In [55]:
from verse.map import LaneMap


class Map2Lanes(LaneMap):
    def __init__(self):
        super().__init__()
        self.add_lanes([lane0, lane1])
        self.h = h
        self.h_exist = h_exist

We can then construct the scenario with two agents and simulate/verify the scenario with map we just created and the result is shown as below. 

In [ ]:
from enum import Enum, auto


class AgentMode(Enum):
    Normal = auto()
    SwitchLeft = auto()
    SwitchRight = auto()


class TrackMode(Enum):
    T0 = auto()
    T1 = auto()
    M01 = auto()
    M10 = auto()


from verse.scenario import Scenario

scenario = Scenario()
scenario.set_map(Map2Lanes())

from tutorial_agent import CarAgent

car1 = CarAgent("car1", file_name="./dl_sec5.py")
car1.set_initial([[0, -0.5, 0, 2], [0.5, 0.5, 0, 2]], (AgentMode.Normal, TrackMode.T0))
car2 = CarAgent("car2", file_name="./dl_sec5.py")
car2.set_initial([[20, -0.5, 0, 1], [20.5, 0.5, 0, 1]], (AgentMode.Normal, TrackMode.T0))
scenario.add_agent(car1)
scenario.add_agent(car2)

traces_simu = scenario.simulate(30, 0.01)
traces_veri = scenario.verify(30, 0.01)

import plotly.graph_objects as go
from verse.plotter.plotter2D import *

fig = go.Figure()
fig = simulation_tree(traces_simu, Map2Lanes(), fig, 1, 2, [1, 2], "lines", "trace")
fig.show()

fig = go.Figure()
fig = reachtube_tree(traces_veri, Map2Lanes(), fig, 1, 2, [1, 2], "lines", "trace")
fig.show()

# Appendix

## Decision Logic Parsing
When writing decision logic, one must consider how their Python code will eventually be parsed into a Verse agent's decision logic. 

One important distinction between Python and Verse logic is that Verse will convert all conditional logic into "if"s. 
For example, this decision logic block written in Python:
````python
if ego.agent_mode != VehicleMode.Accel and other.signal_mode == TLMode.GREEN:
    output.agent_mode = VehicleMode.Accel #accelerate no matter what if green 
elif ego.agent_mode != VehicleMode.Accel and other.signal_mode == TLMode.GREEN: 
    output.agent_mode = VehicleMode.HardBrake #checking if elif works
elif ego.agent_mode != VehicleMode.Accel and other.signal_mode == TLMode.GREEN:
    output.agent_mode = VehicleMode.Normal
else:
    output.agent_mode = VehicleMode.Accel
````
is parsed into the following decision logic in Verse*:
````python
if ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN)):
    output.agent_mode = VehicleMode.Accel #accelerate no matter what if green 
if (((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN)) and (not ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN)))): 
    output.agent_mode = VehicleMode.HardBrake #checking if elif works
if (((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN)) and (not ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN))) and (not ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN)))):
    output.agent_mode = VehicleMode.Normal
if ((not ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN))) and (not ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN))) and (not ((ego.agent_mode != VehicleMode.Accel) and (other.signal_mode == TLMode.GREEN)))):
    output.agent_mode = VehicleMode.Accel
````
*In reality, the Verser parser's decision logic varies a bit more from Python's decision logic. For example, mode names into simply their name, i.e., "MODE" instead of Agent.MODE, and the 'if's are more implicitly encoded as paths within the agent's decision logic.

While this result may seem obvious at first, one can make some important conclusions.

First, 'if-elif-else' and 'if-else' blocks partitions the entire set of input parameters. This result means that the agent must transition somewhere within the block and only transition within one condition, which implies that using 'elif's and 'else's means that one is not leverging the nondeterministic decision logic that Verse allows. 

Second, largely as a consequence of the first conclusion, one must be careful when using 'else' statements. Due to how baseline Verse works, transitions occur instantaneously. Thus, if one lazily writes code that makes the 'else' condition too inclusive, i.e., the 'else' condition transitions the agent state such that in the next decision logic evalution, the 'else' statement is satisfied again, creating a loop. 

An example of this else condition looping can be seen in the code above. If the 'else' condition is fulfilled, the decision logic transitions the agent to the "Accel" state. Once in the "Accel" state, since the earlier 'if's and 'elif's check that the agent is not in the "Accel" mode, the 'else' condition will be fulfilled again, which will happen infinitely.   

If one wants to avoid having to worry about creating such an inclusive 'else' condition, one should just avoid using 'else' statements altogether. 